<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/fitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import pickle

In [ ]:
def fit_naive(Model, Batcher, test_X, loss, initial_epoch, max_epoch):
    X_train, X_test, Y_train, Y_test = Batcher.load_data()
    X = np.concatenate((X_train,X_test)) # 나중에 데이터셋에서 train/test split을 하지 말기. 아니면 test로 지금의 validation을 하고 validation을 진짜 마지막 validation으로 하던가
    Y = np.concatenate((Y_train,Y_test))

    Model.fit(x=X, y=Y, batch_size=100, epochs = max_epoch, initial_epoch=initial_epoch, shuffle=False, validation_data=(test_X,test_X), validation_batch_size=test_X.shape[0])
    # callbacks=[early_stopping, reduce_lr, checkpoint]

In [ ]:
def fit_simMat(Model, Batcher, test_X, loss, initial_epoch, max_epoch):
  X, Y = Batcher.load_data()
  X = np.reshape(X,(-1,128,128,1))
  Y = np.reshape(Y,(-1))
  print(X.shape)
  print(Y.shape)

  # x를 batch 생성 함수로 한다면, batch_size를 사용하지 않고(사이즈는 어짜피 batch 생성 함수에서 결정) steps_per_epoch으로 한 에폭에서 batch 개수를 설정
  # 어떻게 배치까지 다 모여있는 X를 fit에 배치별로 처리하라 입력하지? generator 함수의 형태로 한 배치씩 내놓도록 해야하나.
  Model.fit(x=X, y=Y, batch_size=25, epochs=50, shuffle=False, validation_data=(test_X,test_X), validation_batch_size=test_X.shape[0])

In [ ]:
def fit_simMat_online(Model, Batcher, test_X, loss, initial_epoch, max_epoch):

  
  Model.fit(x=Batcher.train_generator(), steps_per_epoch=10000, epochs=10, shuffle=False, validation_data=(test_X,test_X), validation_batch_size=test_X.shape[0]) # validation_data에서 test_X를 두번 넣는 것은 Y데이터 입력을 위한 dummy. 0짜리 numpy배열을 새로 만들어도 되지만, 이러는 편이 메모리를 덜 쓸 것이라 생각
  # 사실 steps_per_epoch으로 하나의 에폭에 몇개의 배치를 할 지 결정하는건 학습에 아무 상관이 없으나, epoch이 끝날 때 마다 keras가 print하기 때문에 steps_per_epoch을 1로 두고 epoch을 크게 하는 건 출력이 너무 많아 복잡함
  # 그래서 한 epoch에 1000 batch정도로 뒀음

In [ ]:
def fit(Model, Batcher, test_X, loss, initial_epoch, max_epoch):
    if Batcher.name == 'naive_batcher': fit_naive(Model, Batcher, test_X, loss, initial_epoch, max_epoch)
    elif Batcher.name == 'simMat_batcher': fit_simMat(Model, Batcher, test_X, loss, initial_epoch, max_epoch)
    elif Batcher.name == 'simMat_online_batcher': fit_simMat_online(Model, Batcher, test_X, loss, initial_epoch, max_epoch)
        